In [32]:
import os
import json
from termcolor import colored
from llmagent import LLMAgent

In [4]:
def get_tasks(task_split='test', env_type="retail"):
    assert env_type in ["retail", "airline"]
    assert task_split in ["test", "train", "dev"]
    if env_type == "retail":
        match task_split:
            case "test":
                from tau_bench.envs.retail.tasks_test import TASKS_TEST as tasks
            case "train":
                from tau_bench.envs.retail.tasks_train import TASKS_TRAIN as tasks
            case "dev":
                from tau_bench.envs.retail.tasks_dev import TASKS_DEV as tasks
            case _:
                raise ValueError(f"Unknown task split: {task_split}")
    elif env_type == "airline":
        match task_split:
            case "test":
                from tau_bench.envs.airline.tasks_test import TASKS_TEST as tasks
            case "train":
                from tau_bench.envs.airline.tasks import TASKS_TRAIN as tasks
            case _:
                raise ValueError(f"Unknown task split: {task_split}")
    return tasks

In [5]:
tasks = get_tasks()

In [9]:
(tasks[0].instruction)

"You are Yusuf Rossi in 19122. You received your order #W2378156 and wish to exchange the mechanical keyboard for a similar one but with clicky switches and the smart thermostat for one compatible with Google Home instead of Apple HomeKit. If there is no keyboard that is clicky, RGB backlight, full size, you'd go for no backlight. You are detail-oriented and want to make sure everything is addressed in one go."

In [25]:
class TaskGenerator(LLMAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def generate_task(self, instruction):
        system_prompt = f'''
You are a helpful assistant.
The user is provided with an instruction where they are asked to behave as a customer to speak to customer support.
The insrtruction tells the user about the tasks to be solved along with a behavioral instruction about how should they convey the tasks.
However, we wish to extract only the tasks fromt he instruction and ingore all the behavioral part.
Your tasks is to act like a user that ignores all the behavioral instructions and deliver the whole task to the custom,ert support in one go.
Make sure to provide all your details, like name, email, address, zip code, etc. 
Out your response ina  json in the following format:
{{
    task: <task>
}}
'''
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": instruction}]
        res = self.llm_client.complete(model = self.model_name, messages=messages, response_format="json_object").choices[0].message['content']
        res = json.loads(res)
        return res['task']

In [35]:
def convert_all_tasks(task_split='test', env_type="retail", output_file="tasks.json"):
    output_file = f'one_shot_{env_type}_{task_split}.json'
    tasks = get_tasks(task_split, env_type)
    new_tasks = []
    task_generator = TaskGenerator()
    for i in range(len(tasks)):
        print(i)
        print(colored(tasks[i].instruction, 'blue'))
        new_task = task_generator.generate_task(tasks[i].instruction)
        new_tasks.append(new_task)
        print(colored(new_task, 'green'))
    with open(output_file, 'w') as f:
        json.dump(new_tasks, f)
    

In [36]:
convert_all_tasks(task_split='train', env_type="retail")

0
Your name is Omar Anderson and your zip code is 19031. You are logical, independent, relaxing, polite. Return #W6067464 via credit_card_4190576: Electric Kettle; Wall Clock; 
I would like to return order #W6067464, which contains the following items: Electric Kettle and Wall Clock. Please process the refund to my credit card ending in 4190576. My name is Omar Anderson and my zip code is 19031.
1
Your name is Sophia Nguyen and your zip code is 20171. You are confident, organized. Return #W6619432 via paypal_3738584: Dumbbell Set; Yoga Mat; 
I would like to return order #W6619432 via PayPal account paypal_3738584. The items to be returned are: Dumbbell Set and Yoga Mat. My name is Sophia Nguyen and my zip code is 20171.
2
Your name is James Li and your email is james.li4495@example.com. You are rigid, confident, happy, curious, pessimistic. Return #W4435622 via gift_card_1725971: Water Bottle; 
I want to return order #W4435622: Water Bottle using gift card gift_card_1725971. My name is